## MNIST with LeeNet

In [2]:
import torch
import random
import numpy as np
from torch.nn.utils.rnn import pad_sequence
from torch.nn.functional import pad

In [3]:
def fix_seed():
    random.seed(0)
    np.random.seed(0)
    torch.cuda.manual_seed(0)
    torch.backends.cudnn.determenistic=True
    return True

fix_seed()

True

In [6]:
import torchvision.datasets as data

In [7]:
train_data=data.MNIST('./', download=True, train=True)
test_data=data.MNIST('./', download=True, train=False)

X_train, X_test=train_data.train_data.float(), test_data.test_data.float()
y_train, y_test=train_data.targets, test_data.targets

/Users/macbook/miniconda3/envs/torch/lib/python3.7/site-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/Users/macbook/miniconda3/envs/torch/lib/python3.7/site-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


In [8]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([60000, 28, 28]),
 torch.Size([10000, 28, 28]),
 torch.Size([60000]),
 torch.Size([10000]))

In [9]:
X_train = X_train.unsqueeze(1).float()
X_test = X_test.unsqueeze(1).float()

## Define  Architecture of NeuralNet 

In [12]:
class LeNet5(torch.nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        
        self.conv_1 = torch.nn.Conv2d(
            in_channels=1, out_channels=6, kernel_size=5, padding=2
        )
        self.activ_1 = torch.nn.Tanh()
        self.pool_1 = torch.nn.AvgPool2d(kernel_size=2, stride=2)
        
        self.conv_2 = torch.nn.Conv2d(
            in_channels=6, out_channels=16, kernel_size=5, padding=0
        )
        self.activ_2 = torch.nn.Tanh()
        self.pool_2 = toorchAvgPool2d(kernel_size=2, stride=2)
        
        self.fully_conn_1 = torch.nn.Linear(5*5*16, 120)
        self.activ_3 = torch.nn.Tanh()
        
        self.fully_conn_2 = torch.nn.Linear(120, 84)
        self.activ_4 = torch.nn.Tanh()
        
        self.fully_conn_3 = torch.nn.Linear(84, 10)
        
        
    def forward(self, x):
        x = self.conv_1(x)
        x = self.activ_1(x)
        x = self.pool_1(x)
        
        x = self.conv_2(x)
        x = self.activ_2(x)
        x = self.pool_2(x)
        
        x = x.view(x.size(0), x.size(1)*x.size(2)*x.size(3))
        
        x = self.fully_conn_1(x)
        x = self.activ_3(x)
        x = self.fully_conn_2(x)
        x = self.activ_4(x)
        x = self.fully_conn_3(x)
        return x

In [13]:
lenet5 = LeNet5()

NameError: name 'AvgPool2d' is not defined